In [58]:
from ete3 import Tree, TreeStyle
import ast
from datetime import datetime, timedelta
import pandas as pd

In [2]:
plate = 13
date_init = datetime(2020, 7, 1, 19, 57)
dates_datetime = [date_init + timedelta(hours=4) * i for i in range(24)]
dates = [
    f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}'
    for date in dates_datetime
]

In [3]:
def from_connection_tab(connect_tab):
    from_tip = {}
    growth_pattern = {}
    for i in range(len(connect_tab["tip_origin"])):
        tip = connect_tab["tip_origin"][i]
        growth_pattern[tip] = ast.literal_eval(connect_tab["growth_pattern"][i])
        from_tip[tip] = ast.literal_eval(connect_tab["nodes_from_tip"][i])
    return (from_tip, growth_pattern)

In [4]:
from_tip_growth_pattern = []
for date in dates[:-1]:
    from_tip_growth_pattern.append(
        from_connection_tab(pd.read_csv(f"Data/connection_{date}_{plate}.csv"))
    )

In [5]:
connections = [c[0] for c in from_tip_growth_pattern]
growth_patterns = [c[1] for c in from_tip_growth_pattern]

In [6]:
growths = [
    {
        tip: sum([len(branch) for branch in growth_pattern[tip]])
        for tip in growth_pattern.keys()
    }
    for growth_pattern in growth_patterns
]

In [18]:
growing_tips = [
    [node for node in growths[i].keys() if growths[i][node] >= 20]
    for i in range(len(growths))
]

In [24]:
def test(value, listoflist, count):
    counting = 0
    for liste in listoflist:
        if value in liste:
            counting += 1
    return counting >= count

In [20]:
growing_tip_of_interest = [
    tip for tip in growing_tips[0] if test(tip, growing_tips, 10)
]

In [21]:
growing_tip_of_interest

[0, 1, 7, 60, 680, 896, 1971, 1982, 1984]

In [26]:
t = Tree("60;")
for i, connection in enumerate(connections):
    if 60 in connection.keys():
        childs = connection[60]
        for child in childs:
            if child != 60:
                t.add_child(Tree(str(child) + ";"), dist=i)

In [54]:
def make_tree(root, j):
    t = Tree(str(root) + ";")
    for i, connection in enumerate(connections[j:]):
        if root in connection.keys():
            childs = connection[root]
            for child in childs:
                if child != root and test(child, connections[j:], 1):
                    t.add_child(make_tree(child, i), dist=i)
    return t

In [64]:
t = make_tree(60, 0)

In [46]:
t2 = make_tree(7, 0)

In [61]:
ts = TreeStyle()
ts.show_leaf_name = True
ts.mode = "c"
ts.arc_start = -360  # 0 degrees = 3 o'clock
ts.arc_span = 360

In [65]:
t.show(tree_style=ts)